# ROUGE 스코어 이해하기


- 명칭
    - Recall-Oriented : 재현율에 중점을 둔
    - Understudy : 인간을 대신하여
    - for Gisting Evaluation : 핵심 추출 평가

- 개발 목적:
    ROUGE는 원래 텍스트 요약을 평가하기 위해 개발 (Lin, 2004)


- 핵심 개념: "좋은" 요약이란 원본 텍스트의 핵심 정보를 얼마나 잘 포함하느냐!
    - 인간이 작성한 참조 요약과 얼마나 많은 단어나 구가 일치하는지 측정하는 것이 ROUGE의 방식

- ROUGE 사용의 한계

    - 표면적 유사성만 측정: ROUGE는 단어나 구의 겹침만 고려하고 의미적 유사성은 고려X
    - 문맥 이해 부족: 동의어나 환언(paraphrasing)을 인식X
    - 정확성 평가에 한계: 특히 사실적 정확성 평가에는 부적합
    - 창의성 평가에 부적합

- ROUGE 의 발전
    - 초기에는 재현율만을 평가했으나, 이후로 정밀도, f1 스코어까지 확장되었다.
    - ROUGE-L, ROUGE-S, ROUGE-SU 등 다양한 측정 방식으로 확장되었다.
        - ROUGE-L : 가장 긴 공통 부분 수열(LCS)의 재현율을 계산, 연속적이지 않아도 되는 시퀀스를 고려
        - ROUGE-S : 최대 2칸 내에 위치하는 단어 쌍의 재현율을 계산, 연속적이지 않아도 되므로 단어 순서 변화에 덜 민감
        - ROUGE-SU : ROUGE-S의 확장 버전으로, 유니그램을 함께 고려하여 계산, 단어 순서가 완전히 바뀐 경우에도 일부 점수를 부여
        
- 결론:
    ROUGE는 요약 모델 평가에 가장 적합하고 널리 사용되지만, 다른 NLP 작업에도 사용할 수 있다. 다만 그 한계를 인식하고, 필요에 따라 다른 평가 지표와 함께 사용하는 것이 좋다. 최근에는 BERTScore, MoverScore 같은 의미적 유사성을 더 잘 포착하는 지표들도 있다.

In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 29.8 MB/s eta 0:00:00


In [2]:
from collections import Counter
from konlpy.tag import Okt
from nltk.util import ngrams

In [3]:
def tokenize_korean(text):
    """한국어 텍스트를 토큰화합니다."""
    okt = Okt()
    return okt.morphs(text)  # 형태소 단위로 토큰화

def get_ngrams(tokens, n):
    """토큰에서 n-gram을 생성합니다."""
    return list(ngrams(tokens, n))

def count_ngrams(ngram_list):
    """n-gram의 빈도를 계산합니다."""
    return Counter(ngram_list)

In [4]:
reference = "인공지능 기술은 우리의 일상 생활을 변화시키고 있습니다."
candidate = "인공지능 기술이 우리 생활을 크게 변화시키고 있다."

In [5]:
# 토큰화
ref_tokens = tokenize_korean(reference)
cand_tokens = tokenize_korean(candidate)
print(ref_tokens)
print(cand_tokens)

['인공', '지능', '기술', '은', '우리', '의', '일상', '생활', '을', '변화', '시키고', '있습니다', '.']
['인공', '지능', '기술', '이', '우리', '생활', '을', '크게', '변화', '시키고', '있다', '.']


In [6]:
# bi-gram 생성
ref_ngrams = get_ngrams(ref_tokens, 2)
cand_ngrams = get_ngrams(cand_tokens, 2)
print(ref_ngrams)
print(cand_ngrams)

[('인공', '지능'), ('지능', '기술'), ('기술', '은'), ('은', '우리'), ('우리', '의'), ('의', '일상'), ('일상', '생활'), ('생활', '을'), ('을', '변화'), ('변화', '시키고'), ('시키고', '있습니다'), ('있습니다', '.')]
[('인공', '지능'), ('지능', '기술'), ('기술', '이'), ('이', '우리'), ('우리', '생활'), ('생활', '을'), ('을', '크게'), ('크게', '변화'), ('변화', '시키고'), ('시키고', '있다'), ('있다', '.')]


In [7]:
# n-gram 빈도 계산
ref_count = count_ngrams(ref_ngrams)
cand_count = count_ngrams(cand_ngrams)
print(ref_count)
print(cand_count)

Counter({('인공', '지능'): 1, ('지능', '기술'): 1, ('기술', '은'): 1, ('은', '우리'): 1, ('우리', '의'): 1, ('의', '일상'): 1, ('일상', '생활'): 1, ('생활', '을'): 1, ('을', '변화'): 1, ('변화', '시키고'): 1, ('시키고', '있습니다'): 1, ('있습니다', '.'): 1})
Counter({('인공', '지능'): 1, ('지능', '기술'): 1, ('기술', '이'): 1, ('이', '우리'): 1, ('우리', '생활'): 1, ('생활', '을'): 1, ('을', '크게'): 1, ('크게', '변화'): 1, ('변화', '시키고'): 1, ('시키고', '있다'): 1, ('있다', '.'): 1})


In [8]:
# 일치하는 n-gram 계산
matches = 0
for ngram in cand_count:
    m = min(cand_count[ngram], ref_count.get(ngram, 0))
    print(m)

    matches += m
print('총 일치하는 n-gram 개수 :', matches)

1
1
0
0
0
1
0
0
1
0
0
총 일치하는 n-gram 개수 : 4


In [10]:
# @title 정답
def rouge_n(reference, candidate, n):
    """ROUGE-N 점수를 계산합니다.

    Args:
        reference: 참조 텍스트 (문자열)
        candidate: 후보 텍스트 (문자열)
        n: n-gram의 크기 (1 또는 2)

    Returns:
        precision, recall, f1 점수를 포함하는 튜플
    """
    # 토큰화
    ref_tokens = tokenize_korean(reference)
    cand_tokens = tokenize_korean(candidate)

    # n-gram 생성
    ref_ngrams = get_ngrams(ref_tokens, n)
    cand_ngrams = get_ngrams(cand_tokens, n)

    # n-gram 빈도 계산
    ref_count = count_ngrams(ref_ngrams)
    cand_count = count_ngrams(cand_ngrams)

    # 일치하는 n-gram 계산
    matches = 0
    for ngram in cand_count:
        matches += min(cand_count[ngram], ref_count.get(ngram, 0))

    # 정밀도, 재현율, F1 점수 계산
    precision = matches / max(sum(cand_count.values()), 1)
    recall = matches / max(sum(ref_count.values()), 1)

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

def rouge_1(reference, candidate):
    """ROUGE-1 점수를 계산합니다."""
    return rouge_n(reference, candidate, 1)

def rouge_2(reference, candidate):
    """ROUGE-2 점수를 계산합니다."""
    return rouge_n(reference, candidate, 2)

In [11]:
# 예시 1: 유사한 문장
reference1 = "인공지능 기술은 우리의 일상 생활을 변화시키고 있습니다."
candidate1 = "인공지능 기술이 우리 생활을 크게 변화시키고 있어요."

# 예시 2: 요약 예시
reference2 = "대한민국의 수도 서울은 인구 밀도가 높고 다양한 문화를 가진 대도시입니다. 한강이 도시를 가로지르며 많은 관광 명소가 있습니다."
candidate2 = "서울은 인구 밀도가 높은 대한민국의 수도로, 한강이 도시를 관통합니다."

# 예시 3: 뉴스 기사 요약
reference3 = "정부는 오늘 코로나19 방역 지침을 완화하고 사회적 거리두기를 1단계로 하향 조정한다고 발표했다. 이에 따라 다중시설 이용 인원 제한이 해제되고 마스크 착용 의무도 실외에서는 폐지된다."
candidate3 = "정부가 코로나19 방역 지침을 완화하고 사회적 거리두기를 1단계로 낮추기로 했다. 실외 마스크 착용 의무가 없어진다."


In [12]:
reference, candidate = reference1, candidate1

# 토큰화 결과 출력
okt = Okt()
ref_tokens = okt.morphs(reference)
print(f"\n토큰화 예시 (참조 텍스트): {ref_tokens[:10]}...")

# ROUGE-1 계산
r1_precision, r1_recall, r1_f1 = rouge_1(reference, candidate)
print(f"\nROUGE-1 - 정밀도: {r1_precision:.4f}, 재현율: {r1_recall:.4f}, F1: {r1_f1:.4f}")

# ROUGE-2 계산
r2_precision, r2_recall, r2_f1 = rouge_2(reference, candidate)
print(f"ROUGE-2 - 정밀도: {r2_precision:.4f}, 재현율: {r2_recall:.4f}, F1: {r2_f1:.4f}")
print("-" * 80)


토큰화 예시 (참조 텍스트): ['인공', '지능', '기술', '은', '우리', '의', '일상', '생활', '을', '변화']...

ROUGE-1 - 정밀도: 0.7500, 재현율: 0.6923, F1: 0.7200
ROUGE-2 - 정밀도: 0.3636, 재현율: 0.3333, F1: 0.3478
--------------------------------------------------------------------------------
